In [6]:
#### Dependencies

from PIL import Image
from skimage.feature import greycomatrix, greycoprops

import glob
import math
import matplotlib.pyplot as plt
import os
import random


In [7]:
#### Global variables

training_data = []
validation_data = []
class_names = []
map_8bit_to_3bit = [i // 32 for i in range(256)]


In [12]:
#### Functions

def load_img(img_path):
    return Image.open(img_path).convert('L')

def get_img_size(img):
    return img.size

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def get_resized_img(img, dimension):
    return img.resize(dimension)
    
def get_img_colors(img):
    return list(img.getdata())

def get_3bit_img_colors(img):
    img_colors = get_img_colors(img)
    
    loop_count = 0
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = []
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix

def get_img_features(img):
    img_3bit_colors = get_3bit_img_colors(img)
    img_matrix = get_img_matrix(img_3bit_colors)
    
    glcm_matrix = greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    glcm_components = ['contrast', 'correlation', 'energy', 'homogeneity', 'ASM', 'dissimilarity']
    
    img_features = []
    for glcm_component in glcm_components:
        img_features.append(greycoprops(glcm_matrix, glcm_component)[0][0])
        
    return tuple(img_features)

# Modelling

def init_model():
    global training_data
    global validation_data
    
    training_data = []
    validation_data = []
    
def load_class_names(training_path):
    global class_names
    
    class_names = [class_name for class_name in os.listdir(training_path)]
            
def insert_img_features_into_training_data(class_name, img_features):
    global training_data
    
    row_tuple = (class_name,) + img_features
    training_data.append(row_tuple)
        
def insert_img_features_into_validation_data(class_name, img_features):
    global validation_data
    
    row_tuple = (class_name,) + img_features
    validation_data.append(row_tuple)

def load_preprocessed_img(img_path, dimension=(128, 128)):
    img = load_img(img_path)
    img = get_resized_img(img, dimension)
    
    return img
        
def load_data(training_path, validation_path, training_data_path, validation_data_path, img_type='*.jpg'):
    global training_data
    global validation_data
    
    print('Loading data...')
    
    if(os.path.exists(training_data_path) and os.path.exists(validation_data_path)):
        training_data_file = open(training_data_path, 'r')
        training_data_str = training_data_file.read().split('\n')
        
        for row in training_data_str:
            row = row[1:len(row) - 1].split(', ')
            row[1:] = [float(value) for value in row[1:]]
            row_tuple = tuple(row)
            training_data.append(row_tuple)
        
        validation_data_file = open(training_data_path, 'r')
        validation_data_str = validation_data_file.read().split('\n')
        
        for row in validation_data_str:
            row = row[1:len(row) - 1].split(', ')
            row[1:] = [float(value) for value in row[1:]]
            row_tuple = tuple(row)
            validation_data.append(row_tuple)
    else:
        for class_name in class_names:
            img_paths = glob.glob(training_path + class_name + '/' + img_type)

            for img_path in img_paths:
                img = load_preprocessed_img(img_path)
                img_features = get_img_features(img)

                insert_img_features_into_training_data(class_name, img_features)

        for class_name in class_names:
            img_paths = glob.glob(validation_path + class_name + '/' + img_type)

            for img_path in img_paths:
                img = load_preprocessed_img(img_path)
                img_features = get_img_features(img)

                insert_img_features_into_validation_data(class_name, img_features)
                
        data_path = training_data_path[:len(training_data_path) - training_data_path[::-1].index('/')]
        if(not os.path.exists(data_path)):
            os.mkdir(data_path)
                
        with open(training_data_path, 'w+') as file_writer:
            training_data_last_index = len(training_data) - 1
            for row_index in range(training_data_last_index):
                file_writer.write(str(training_data[row_index]).replace('\'', '') + '\n')
            file_writer.write(str(training_data[training_data_last_index]).replace('\'', ''))
                
        with open(validation_data_path, 'w+') as file_writer:
            validation_data_last_index = len(validation_data) - 1
            for row_index in range(validation_data_last_index):
                file_writer.write(str(validation_data[row_index]).replace('\'', '') + '\n')
            file_writer.write(str(validation_data[validation_data_last_index]).replace('\'', ''))
            
    print('--> Done')
    
def train(training_rate=0.8):
    print('Training...')
    training_data_sample = []
    testing_data_sample = []
    
    for row in training_data:
        random_splitter = random.uniform(0, 1)

        if(random_splitter <= training_rate):
            training_data_sample.append(row)
        else:
            testing_data_sample.append(row)
            
    print('--> Done')
    return training_data_sample, testing_data_sample
    
def get_euclidean_distance(model_img_features, img_features):
    distance = 0.0
    
    for glcm_index in range(6):
        distance += (model_img_features[glcm_index] - img_features[glcm_index]) ** 2
        
    return math.sqrt(distance)

def get_img_features_class(img_features, training_data_sample, k_neighbors=1):
    minimum_training_img_distances = [-1 for i in range(k_neighbors)]
    minimum_training_img_class_names = ['unknown' for i in range(k_neighbors)]
    is_first_loop = True
    for row in training_data:
        training_img_class_name = row[0]
        training_img_features = row[1:]
        
        euclidean_distance = get_euclidean_distance(training_img_features, img_features)
            
        if(is_first_loop):
            minimum_training_img_distances[0] = euclidean_distance
            minimum_training_img_class_names[0] = training_img_class_name
            is_first_loop = False
        else:
            for i in range(k_neighbors):
                if(euclidean_distance < minimum_training_img_distances[i] or minimum_training_img_distances[i] == -1):
                    for j in range(k_neighbors - 1, i, -1):
                        minimum_training_img_distances[j] = minimum_training_img_distances[j-1]
                        minimum_training_img_class_names[j] = minimum_training_img_class_names[j-1]
                    minimum_training_img_distances[i] = euclidean_distance
                    minimum_training_img_class_names[i] = training_img_class_name
                    break
    
    minimum_training_img_class_names = list(filter(lambda val: val != 'unknown', minimum_training_img_class_names))
    img_class_name = max(set(minimum_training_img_class_names), key=minimum_training_img_class_names.count)
    
    return img_class_name
    
def get_img_class(img, training_data_sample, k_neighbors=1):
    img_features = get_img_features(img)
    img_class_name = get_img_features_class(img_features, training_data_sample, k_neighbors)
    
    return img_class_name
    
def test(testing_data_sample, training_data_sample, k_neighbors=1):
    print('Testing...')
    total_correct_answer = 0
    total_guess = 0
    
    for row in testing_data_sample:
        expected_class_name = row[0]
        test_img_features = row[1:]
        
        test_img_class_name = get_img_features_class(test_img_features, training_data_sample, k_neighbors)
            
        if(expected_class_name == test_img_class_name):
            total_correct_answer += 1
                
        total_guess += 1
    
    accuracy = (total_correct_answer / total_guess) * 100
    
    print('--> Done, accuracy = ' + str(accuracy) + ' %')
    
    return accuracy
    
def validate(training_data_sample, k_neighbors=1):
    print('Validating...')
    total_correct_answer = 0
    total_guess = 0
    
    for row in validation_data:
        expected_class_name = row[0]
        validation_img_features = row[1:]
        
        img_class_name = get_img_features_class(validation_img_features, training_data_sample, k_neighbors)
            
        if(expected_class_name == img_class_name):
            total_correct_answer += 1
                
        total_guess += 1
            
    print('--> Done')
    
    accuracy = (total_correct_answer / total_guess) * 100
    
    return accuracy
    

In [21]:
#### Main

# Defining model name
model_name = 'cap_tulis_glcm_knn'

# Defining paths
root_path            = './'
test_path            = root_path + 'test/'
training_path        = root_path + 'training/'
validation_path      = root_path + 'validation/'
data_path            = root_path + 'data/'
training_data_path   = data_path + model_name + '_training.data'
validation_data_path = data_path + model_name + '_validation.data'

# Props
iterations = 3
epochs = 5
training_rate = 0.8
img_type = '*.jpg'
k_neighbors = 5

# Init
init_model()
load_class_names(training_path)
load_data(training_path, validation_path, training_data_path, validation_data_path, img_type)

for iteration in range(iterations):
    print('=========================================')
    print('Iteration ' + str(iteration + 1))
    print('=========================================')
    training_data_sample = []
    testing_data_sample  = []
    
    for epoch in range(epochs):
        print('Epoch ' + str(epoch + 1))
        new_training_data_sample, new_testing_data_sample = train(training_rate)
        training_data_sample += new_training_data_sample
        testing_data_sample += new_testing_data_sample
        test(testing_data_sample, training_data_sample, k_neighbors)
        print()

    accuracy = validate(training_data_sample, k_neighbors)

    print('Accuracy: ' + str(accuracy) + ' %')
    print('\n\n')

Loading data...
--> Done
Iteration 1
Epoch 1
Training...
--> Done
Testing...
--> Done, accuracy = 91.66666666666666 %

Epoch 2
Training...
--> Done
Testing...
--> Done, accuracy = 93.56223175965665 %

Epoch 3
Training...
--> Done
Testing...
--> Done, accuracy = 94.42815249266863 %

Epoch 4
Training...
--> Done
Testing...
--> Done, accuracy = 93.97321428571429 %

Epoch 5
Training...
--> Done
Testing...
--> Done, accuracy = 94.26523297491039 %

Validating...
--> Done
Accuracy: 94.40298507462687 %



Iteration 2
Epoch 1
Training...
--> Done
Testing...
--> Done, accuracy = 93.85964912280701 %

Epoch 2
Training...
--> Done
Testing...
--> Done, accuracy = 94.27312775330397 %

Epoch 3
Training...
--> Done
Testing...
--> Done, accuracy = 94.95268138801262 %

Epoch 4
Training...
--> Done
Testing...
--> Done, accuracy = 94.73684210526315 %

Epoch 5
Training...
--> Done
Testing...
--> Done, accuracy = 94.90566037735849 %

Validating...
--> Done
Accuracy: 94.40298507462687 %



Iteration 3
Epoch 1

In [22]:
img = load_img(test_path + 'cap.jpg')
print(get_img_class(img, 3))
# test correct

cap


In [23]:
img = load_img(test_path + 'cap2.jpg')
print(get_img_class(img, 3))
# test false

tulis


In [24]:
img = load_img(test_path + 'tulis.jpg')
print(get_img_class(img, 3))
# test false

cap
